In [56]:
# импорты
import nltk
import re
import pickle
from string import punctuation
import math
from tqdm import tqdm_notebook
import csv
import gensim
import random

import pymystem3
m = pymystem3.Mystem() #для использования лемматизации

import pymorphy2
morph = pymorphy2.MorphAnalyzer()

import pandas as pd

'''from nltk.corpus import stopwords
stop_words = stopwords.words("russian")'''

'from nltk.corpus import stopwords\nstop_words = stopwords.words("russian")'

### Посчитать информативность слов из б1 и меньше - скачать списки, используемые в РКИ. Исходя из этого вычислять сложность

In [2]:
def unpack(data):
    input = open(data, 'rb')
    obj = pickle.load(input)
    input.close()
    return obj

# загружаем частоты лем униграмм
unigrams = unpack('1stemgrams.data')
# убрали пробел из начала слов
unigrams = {w[1:]:f for w,f in unigrams.items()}

In [3]:
bigrams = unpack('2grams.data')
# убрали пробел из начала слов, пунктуацию (кроме дефисов) и двойные пробелы
bigrams  = {''.join([i for i in w[1:] if i not in punctuation.replace('-','')]).replace('  ',' '):f for w,f in bigrams.items()}

In [4]:
list(bigrams.keys())[:10]

['дальнейший допрос',
 'оканчивать преступление',
 'развитие христианство',
 'и затонуть',
 'подразумевать она',
 'и однозначный',
 '2 введение',
 'желать услышать',
 'являться недооценка',
 'закрепляться по']

In [5]:
trigrams = unpack('3grams.data')
# убрали пробел из начала слов, пунктуацию (кроме дефисов) и двойные пробелы
trigrams = {''.join([i for i in w[1:] if i not in punctuation.replace('-','')]).replace('  ',' '):f for w,f in trigrams.items()}

In [6]:
list(trigrams.keys())[:10]

['в лифт я',
 'что сам русский',
 'почему ты думать',
 'фестиваль в кольмар',
 'твердость и прочность',
 'тот число руководитель',
 'судьба решать иначе',
 'мысль рождаться в',
 'мой учение то',
 'да только что']

In [7]:
# читает из файла, убирает двойные пробелы и ручные переносы, последний \n
def reading(file):
    f = open('texts\\{}.txt'.format(file), 'r', encoding='utf-8')
    text = f.read()
    text = text.replace('  ', ' ')
    text = text.replace('-\n', '')
    if text[-1] == '\n': # убираем последний \n, если такой есть 
        text = text[:-1]
    f.close()
    return text

In [8]:
def minimummaker(): #ф-ция превращения текстового файла с минимумом в питоновский список. Запаковка списка
    with open('min.txt', encoding='utf-8') as file:
        lemtokens = [morph.parse(i)[0].normal_form for i in re.findall('\w+', file.read().lower())] #делаем список лемм слов
        minimum = list(set(lemtokens)) #убираем повторы
        output = open('minimum.pkl', 'wb')
        pickle.dump(minimum, output, 2)
        output.close()

def loadminimum(): #распаковка cписка с минимумом. Возвращает неупорядоченный список
    input = open('minimum.pkl', 'rb')
    minimum = pickle.load(input)
    input.close()
    return minimum

#minimummaker()
minimum = loadminimum()
#print(minimum[-50:]) #проверка работы
#print(len(minimum)) #2549 слов в списке

In [33]:
df = pd.read_csv('texts1.csv', sep='\t', encoding='utf-8')

In [36]:
df.drop(['Unnamed: 0'],axis=1)

,headlines,texts,rubrics,links
0,Трамп назвал главу Нью-Йорка худшим мэром в США,Президент США Дональд Трамп назвал главу Нью-Й...,world,https://news.yandex.ru//story/Tramp_nazval_gla...
1,В Германии недовольные «Северным потоком — 2» ...,Протестующие против строительства «Северного п...,world,https://news.yandex.ru//story/V_Germanii_nedov...
2,США внесут Huawei в черный список,"Американский Минторг объявил, что внесёт крупн...",world,https://news.yandex.ru//story/SSHA_vnesut_Huaw...
3,Мадуро прокомментировал решение США запретить ...,"Президент Венесуэлы Николас Мадуро заявил, что...",world,https://news.yandex.ru//story/Maduro_prokommen...
4,Под завалами рухнувшего в Шанхае здания погибл...,Спасателям удалось обнаружить и вытащить из по...,world,https://news.yandex.ru//story/Pod_zavalami_ruk...
5,Появились тизерные изображения новой BMW 1-Series,Новый BMW 1-Series разработан на базе платформ...,world,https://news.yandex.ru//story/Poyavilis_tizern...
6,Philips перешла в Нидерландах на возобновляему...,Нидерландские предприятия Royal Philips NV пол...,world,https://news.yandex.ru//story/Philips_pereshla...
7,Виновницей крупнейшего в Калифорнии пожара наз...,Представители калифорнийского управления лесно...,world,https://news.yandex.ru//story/Vinovnicej_krupn...
8,Рождаемость в США достигла минимума,"По данным экспертов, снижение рождаемости набл...",world,https://news.yandex.ru//story/Rozhdaemost_v_SS...
9,В Турции анализ ДНК выявил новую тайну древней...,Международная группа генетиков и археологов из...,world,https://news.yandex.ru//story/V_Turcii_analiz_...


In [ ]:
def measure potential(n = 5, base = ['yarn','model','asis'][0]):
    
    def make_sample(n):
        rubric = list(set(df['rubrics']))
        text_rubrics = df.filter(items=['texts', 'rubrics'])
        text_to_potential = []
        for r in rubric:
            rub_texts = []
            for l in range(len(text_rubrics)):
                if text_rubrics.loc[l][1]==r:
                    rub_texts.append(text_rubrics.loc[l][0])
            text_to_potential.append(random.sample(rub_texts,n))
        return text_to_potential
    
    text_to_potential = make_sample(n)
    
    positif = 0
    total = len(text_to_potential)
    
    for text in text_to_potential:
        

In [60]:
# всякие непонятные куски, портящие статистику по минимумам:
to_remove = ['ми', 'вод', 'кий', 'ре','вая', 'ча', 'то быть', 'чка', 'стен']
with open('B1.txt', 'r', encoding='utf-8') as file:
    lemtokens = [morph.parse(i)[0].normal_form for i in file.read().lower().split('\n') if morph.parse(i)[0].normal_form not in to_remove] #делаем список лемм слов
    minimum = list(set(lemtokens)) #убираем повторы

In [61]:
print(minimum[-50:])

['пессимист', 'постараться ', 'серебро ', 'лёгкий', 'бой', 'зарубежный', 'стоить ', 'сфотографировать ', 'учиться ', 'экспорт ', 'сейчас', 'приехать ', 'бензин', 'момент', 'мороз', 'шуметь ', 'надежда', 'чувствовать ', 'проверить ', 'вырасти ', 'представитель', 'салют', 'церковь', 'таблетка', 'такси ', 'загорать ', 'мы', 'вести ', 'чтобы', 'бог', 'семья', 'тихо', 'жаль ', 'самостоятельно', 'небо', 'выражать ', 'навстречу ', 'фестиваль', 'выполнять', 'приглашать ', 'должный', 'жестокий', 'по-твоему', 'считать ', 'сдать ', 'эксперимент', 'официальный', 'никакой', 'модель ', 'посольство']


In [17]:
# штука, которая рассчитывает порог коэффициента информативности слова на основе лексического минимума - Я ПРИДУМАЛА!
# отрицательный. чем коэффициент информативности меньше, чем сложнее слово (с уменьшением дроби логарифм уменьшается тоже)
from collections import Counter

det = sum(f for f in unigrams.values())+1
inf_coef = Counter([round(math.log((unigrams.get(w)+1)/det),1) for w in minimum if w in unigrams])

In [18]:
l = []
for w in minimum:
    if w in unigrams:
        if round(math.log((unigrams.get(w)+1)/det),1)<-17:
            l.append(w)

In [19]:
print(l)

['съесть']


In [20]:
import numpy as np
np.median(list(inf_coef.keys()))

-9.2

In [21]:
inf_coef.most_common(10)

[(-9.3, 32),
 (-9.0, 24),
 (-8.9, 21),
 (-9.2, 20),
 (-9.4, 20),
 (-8.6, 19),
 (-8.8, 18),
 (-9.7, 18),
 (-9.8, 18),
 (-9.6, 17)]

In [22]:
# загрузка модели
def model_loading(file):
    model = gensim.models.KeyedVectors.load_word2vec_format(file, binary=False)
    model.init_sims(replace=True)
    print('Done!') 
    return model

model = model_loading('news_upos_cbow_600_2_2018.vec') #на загрузку тратится минут 10

Done!


In [10]:
# загрузка словаря ASIS
with open('syns.data', 'rb') as f:
     asis = pickle.load(f)

In [23]:
# Таблица конверсии в UPoS из тэгов Mystem
# словарь, переводящий теги mystem в universal теги моделей
mystem_tags = {'A' : 'ADJ',
       'ADV' : 'ADV',
       'ADVPRO' : 'ADV',
       'ANUM' : 'ADJ',
       'APRO' : 'DET',
       'COM' : 'ADJ',
       'CONJ' : 'SCONJ',
       'INTJ' : 'INTJ',
       'NONLEX' : 'X',
       'NUM' : 'NUM',
       'PART' : 'PART',
       'PR' : 'ADP',
       'S' : 'NOUN',
       'SPRO' : 'PRON',
       'UNKN' : 'X',
       'V' : 'VERB',
       'X' : 'X',
      'PROPN' : 'PROPN'} #последних 2 тегов в майстеме нет, но они задаются в классе для слов в соответсвующими пометами

# словарь, переводящий теги пайморфи в universal теги моделей
pymorphy_tags = {'ADJF':'ADJ',
    'ADJS' : 'ADJ',
    'ADVB' : 'ADV',
    'COMP' : 'ADV',
    'GRND' : 'VERB',
    'INFN' : 'VERB',
    'NOUN' : 'NOUN',
    'PRED' : 'ADV',
    'PRTF' : 'ADJ',
    'PRTS' : 'VERB',
    'VERB' : 'VERB'}

In [46]:
class Token():
    def __init__(self, w):
        
        self.num = None # номер в тексте
        self.complexity = None # сложность слова
        self.av_similarity = None
        
        # три варианта инициализации: 
        ## из анализа текста, 
        ## из уже имеющегося объекта (для дочернего класса ComplexWord) 
        ## из строки
        
        if isinstance(w, dict): # если получили результат работы mystem
            
            self.text = w['text']  # сам токен
            self.len = len(w['text']) # его длина
            
            # определяет, сделан ли анализ и, соответственно, рассматривать ли как слово, требующее упрощения
            gram = w.get('analysis')
            if gram:
                self.lexem = gram[0]['lex']  # лемма
                
                if not self.named_entity(gram[0]):  # именованная сущность или нет
                    self.pos = self.pos_tag(gram[0]['gr'])  # часть речи
                else:
                    self.pos = 'PROPN' # universal tag for named entity - у майстема таких нет
                
                    
            elif any(p in w['text'] for p in punctuation+'–«»'): # если это знак пунктуации (может быть с пробелом!)
                self.lexem = '_PUNKTUATION_'
                self.pos = None
            
            elif not re.findall('\S',w['text']): # если это только пробельные символы
                self.lexem = '_SPACE_'
                self.pos = None
                
            # остальное - неизвестная и ненужная ерунда?
            else:
                self.lexem = '_UNK_'
                self.pos = 'X' # universal tag for unknown
            
            
        elif isinstance(w, Token): # для определения объектов дочернего класса ComplexWord
            self.text = w.text
            self.num = w.num
            self.lexem = w.lexem 
            self.len = w.len
            self.pos = w.pos
            self.complexity = w.complexity
            
        
        elif isinstance(w, str): # если хотим как класс токен определить строку, полученную из словаря или модели
            self.text = w
            self.pos = None
            self.lexem = w
            self.len = len(w)
            self.num = None
            self.complexity = None
            
        
    # вытаскивает часть речи из разбора майстем
    def pos_tag(self,gram):
        if ',' in gram:
            gram = gram.split(',')[0]
        if '=' in gram:
            gram = gram.split('=')[0]
        return gram
        
    # определяет по тегам, является ли именованной сущностью
    def named_entity(self,gram):
        markers = {'сокр': ' - сокращение', 'фам': ' - фамилия', 'имя': ' - имя собственное', 'гео': ' - название места', }
        if any(m in gram['gr'] for m in markers.keys()):
            return True
        else:
            return False

    def complexity_params(self, param = 'freq'):
        # если по частотности
        if param == 'freq':
            self.complexity = unigrams.get(self.lexem, 0)
            
        # если по коэффициенту информативности. Отрицательное значение. Чем он меньше, тем сложнее
        elif param == 'inf':
            self.complexity = math.log((unigrams.get(self.lexem, 0)+1)/(sum(f for f in unigrams.values())+1))
        
    
    def is_complex(self, threshold = '600', use_min = False, len_threshold = 1000):
        exceptions = ['_PUNKTUATION_', '_SPACE_', '_UNK_']
        # проверка, что это слово и что его нужно рассматривать как сложное (не нарицательное)
        if not any(exception in self.lexem for exception in exceptions) and self.pos not in ['PROPN']:
            
            # если показатель сложности - вхождение в минимум
            if use_min:
                if self.lexem not in minimum:
                    return True
                else:
                    return False

            # если показатель сложности - пороговое значение сложности
            # также может использоваться длина. По умолчанию слишком большое - 1000 (т.е. этот параметр не учитывается)
            else:
                if self.complexity < float(threshold) or self.len > len_threshold:
                    return True
                else:
                    return False
        else:
            return False
    
    def convert_universal(self):
        if self.pos in mystem_tags:
            self.pos = mystem_tags[self.pos]
        else:
            self.pos = 'X' # Х - universal тег для неизвестных слов
        return self
    
    # нужна в двух случаях: для Замен, полученных из словарей, чтобы приписывать им соответствующий параметр,
    # и для подсчета адекватности слова контексту
    def cos_sim(self, context):
        #print(self.lexem, target.lexem, self.pos, target.pos)
        subst_query = str(self.lexem+'_'+self.pos)
        target_query = str(context.lexem+'_'+context.pos)
        if subst_query in model and target_query in model:
            return model.similarity(subst_query, target_query)
        else:
            return 0.0
    
'''Пример вывода атрибутов объектов класса 
token.num, token.text, token.lem, token.pos, token.complexity, token.is_complex(threshold, use_min):

223 Кукушкин _NAMED_ENTITY_ S -19.06687873346149 False
224   _SPACE_ None -19.06687873346149 False
225 является являться V -7.502177090710664 False
226   _SPACE_ None -19.06687873346149 False
227 должником должник S -11.184186527172464 True
228   _SPACE_ None -19.06687873346149 False
229 банка банк S -9.300701131889967 False
230 .  _PUNKTUATION_ None -19.06687873346149 False
'''

'Пример вывода атрибутов объектов класса \ntoken.num, token.text, token.lem, token.pos, token.complexity, token.is_complex(threshold, use_min):\n\n223 Кукушкин _NAMED_ENTITY_ S -19.06687873346149 False\n224   _SPACE_ None -19.06687873346149 False\n225 является являться V -7.502177090710664 False\n226   _SPACE_ None -19.06687873346149 False\n227 должником должник S -11.184186527172464 True\n228   _SPACE_ None -19.06687873346149 False\n229 банка банк S -9.300701131889967 False\n230 .  _PUNKTUATION_ None -19.06687873346149 False\n'

In [47]:
'''# вытаскивает часть речи из разбора майстем
def pos(gram):
    if ',' in gram:
        gram = gram.split(',')[0]
    if '=' in gram:
        gram = gram.split('=')[0]
    return gram
'''

#анализ текста 
def text_structuring(text, param, threshold, use_min):
    # анализирует текст 
    analysis = m.analyze(text)
    tokens = []
    for i, w in enumerate(analysis): # состаляем список объектов Tokens
        token = Token(w)
        token.num = i # добавляем токену в атрибуты его номер в тексте
        token.complexity_params(param) # переопределяем сложность на основе выбранного параметра
        token.convert_universal() # превращаем POS в universal формат
        print(token.num, token.text, token.lexem, token.pos, token.complexity, token.is_complex(threshold, use_min))
        tokens.append(token)
    return tokens

In [48]:
# ПАРАМЕТРЫ анализа слов
complexity_type = 'inf'
#global_threshold = -8.5
global_threshold = np.median(list(inf_coef.keys()))
use_min=False

In [49]:
text = reading('news2')
tokens = text_structuring(text, complexity_type, global_threshold, use_min)

0 Бракованные бракованный ADJ -14.010632928113182 True
1   _SPACE_ X -19.06687873346149 False
2 белорусские белорусский ADJ -11.414807987345007 True
3   _SPACE_ X -19.06687873346149 False
4 лекарства лекарство NOUN -10.277066347270518 True
5   _SPACE_ X -19.06687873346149 False
6 изъяли изымать VERB -11.391796875745156 True
7   _SPACE_ X -19.06687873346149 False
8 из из ADP -5.430146815838169 False
9   _SPACE_ X -19.06687873346149 False
10 обращения обращение NOUN -9.762410698760608 True
11 
 _SPACE_ X -19.06687873346149 False
12 
 _SPACE_ X -19.06687873346149 False
13 Лекарственные лекарственный ADJ -11.538009476819239 True
14   _SPACE_ X -19.06687873346149 False
15 препараты препарат NOUN -10.247213384120835 True
16   _SPACE_ X -19.06687873346149 False
17 из из ADP -5.430146815838169 False
18   _SPACE_ X -19.06687873346149 False
19 Белоруссии белоруссия PROPN -11.274942776523432 False
20 ,  _PUNKTUATION_ X -19.06687873346149 False
21 в в ADP -3.486155131943022 False
22   _SPACE_ X -1

In [50]:
class Substitution(Token):
    def __init__(self, w):
        super().__init__(w)
        self.similarity = None
        self.fitness = None
        self.closeness = None
    
    # для слов из словаря и тезауруса: определяем тег пайморфи, переводим в формат universal - так быстрее, чем майстемом
    def tagging(self, w):
        tag = morph.parse(w)[0].tag.POS
        if tag in pymorphy_tags:
            return pymorphy_tags[tag]
        else:
            return 'X' # Х - universal тег для неизвестных слов
    
    # приписываем недостающие атрибуты словам, взятым из словаря или тезауруса
    def setting_atr(self, target):
        self.pos = self.tagging(self.lexem)
        self.complexity_params(complexity_type)
        self.similarity = self.cos_sim(target)

In [51]:
class Complex_word(Token):
    def __init__(self, w):
        super().__init__(w)
        self.substituts = None
        self.place = None
        self.context = None
        self.easier = []
        
    # список замен, в зависимости от выбранной базы
    def search_substituts(self, base_type='model'):
        
        # поиск по модели
        def model_search(lexem, pos):
            query = str(lexem+'_'+pos)
            #print(query)
            if query in model:
                # формируем список квазисинонимов той же части речи
                # при этом превращаем их в объекты соответствующего класса
                syn_tokens = []
                for syn, sim in model.most_similar(positive=[query]):
                    syn_text = syn[:syn.find('_')] # текст до части речи
                    
                    syn_tok = Substitution(syn_text) # из текстовой строки инициализируем объект класса
                    syn_tok.pos = syn[syn_tok.len+1:] # часть речи
                    
                    syn_tok.complexity_params(complexity_type) # сложность по функции в зависимости от выбранного параметра
                    
                    syn_tok.similarity = sim # а близость по параметру модели
                    
                    syn_tokens.append(syn_tok)
                    
                return syn_tokens
            else:
                return [] 

        # поиск по YARN
        def yarn_search(target, filepath = 'yarn-synsets1.csv'):
            with open(filepath, "r", newline="") as file: # постепенный просмотр файла с синсетами (множествами синонимов)
                reader = csv.DictReader(file, delimiter=';')
                lst = []
                for i,row in enumerate(reader):
                    cur_line = row['words'].split(';') # считываем колонку с синсетами
                    if len(cur_line)>1:
                        if target.lexem in cur_line:
                            del(reader)
                            for c in cur_line:
                                if ' ' not in c and c!=target.lexem: # формируем список однословных синонимов
                                    sub_tok = Substitution(c)
                                    
                                    sub_tok.setting_atr(target)
                                    
                                    if sub_tok not in lst:
                                        lst.append(sub_tok) 
                            #TODO: выделить неоднословные в отдельный класс и поискать их частотность по n-граммам?
                            break
                #print(lst)
                return lst 
        
        #поиск по ASIS
        def asis_search(target):
            if target.lexem in asis:
                lst = []
                for s in asis[target.lexem]:
                    if ' ' not in s: # формируем список однословных синонимов
                        sub_tok = Substitution(s)
                        sub_tok.setting_atr(target)
                        lst.append(sub_tok)
                return lst
            else:
                return []

        
        if base_type == 'model':
            self.substituts = model_search(self.lexem, self.pos)
            
        if base_type == 'yarn':
            self.substituts = yarn_search(self)
        
        if base_type == 'asis':
            self.substituts = asis_search(self)
            
        return self
    
    def find_easier(self, use_min = False, threshold = global_threshold):
        for sub in self.substituts:
            if not sub.is_complex(threshold = threshold, use_min = use_min):
                if sub.complexity > self.complexity:
                    self.easier.append(sub)
        return self
    
    def make_window(self, tokens, window = 10):
        context = [self]
        left_ind = self.num-1
        right_ind = self.num+1
        ind = 0
        # добавляем по одному слову слева и/или справа, пока не наберется window + само слово
        while len(context)<window+1:
            while left_ind >= 0:
                left_w = tokens[left_ind]
                left_ind-=1
                # проверка, что это слово
                if any(exception in left_w.lexem for exception in ['_PUNKTUATION_', '_SPACE_', '_UNK_']):
                    continue
                else:
                    context[:0] = [left_w] #вставляем слово слева от цепочки
                    ind+=1 # индекс слова сдвигается
                    break

            while right_ind < len(tokens):
                right_w = tokens[right_ind]
                right_ind+=1
                # проверка, что это слово и что его нужно рассматривать как сложное (не нарицательное)
                if any(exception in right_w.lexem for exception in ['_PUNKTUATION_', '_SPACE_', '_UNK_']):
                    continue
                else:
                    context.append(right_w) # справа от цепочки
                    break
        self.place = ind
        self.context = context
        return self

In [52]:
# Отбор сложных слов, превращение их в подкласс сложных слов и поиск замен
# несложные слова также остаются на своих местах
def selecting_complex(tokens, base_type=['yarn','model','asis'][0], threshold = global_threshold, use_min = False):
    complex_words = []
    for token in tokens:
        if token.is_complex(threshold, use_min):
            comp_token = Complex_word(token) # токен становится Сложным словом
            
            comp_token.search_substituts(base_type=base_type)
            
            complex_words.append(comp_token)
            
            # код для принтов
            
            '''
            if comp_token.substituts:
                for syn in comp_token.substituts:
                    print (comp_token.lexem, comp_token.complexity, base_type, ':', syn.lexem, syn.complexity, syn.similarity)
            '''
            comp_token.find_easier(threshold, use_min)
            if comp_token.easier:
                for syn in comp_token.easier:
                    print (comp_token.lexem, comp_token.pos, base_type, ':', syn.lexem, syn.pos, syn.complexity, syn.similarity)
            
        else:
            complex_words.append(token)
    return complex_words

complex_words = selecting_complex(tokens, 'yarn', threshold = global_threshold, use_min = False)

изымать VERB yarn : брать VERB -8.611951584274115 0.10304389896830814
пациент NOUN yarn : больной ADJ -8.563840649103 0.3274560641921585
выводить VERB yarn : открывать VERB -8.24556163363519 0.2906104975356157
осложнение NOUN yarn : проблема NOUN -8.221393716707802 0.18532289510563849
осложнение NOUN yarn : задача NOUN -8.337681827282445 0.016228775087925692
пациент NOUN yarn : больной ADJ -8.563840649103 0.3274560641921585


### При use_min = True в основном возвращает пустоту - очень мало там слов

### теперь, когда для каждого сложного слова есть список его синонимов с нужными параметрами, можно делать модель на н-граммах
### контекстное окно: 2n+1 слов (по n слева и справа), можно задавать. Вероятность на основе логарифма + Лапласса

In [36]:
from nltk.util import ngrams

In [37]:
ngrams_dict = {0: {}, 1: unigrams, 2: bigrams, 3: trigrams}

### Вообще должно быть не деление на V, а деление на объем корпуса - количество вхождений слов. Те подсчитать сумму значений частот униграмм

In [55]:
def ngram_prob(ngram, n, use_log= False):
    c1 = ' '.join(ngram[-n:]) # в числителе частота строки длины n
    c2 = ' '.join(ngram[-n:-1]) # в знаменателе - строки без последнего символа
    d = ngrams_dict.get(n) # для поиска числителя берем словарь n-грамм
    d2 = ngrams_dict.get(n-1) # для поиска знаменателя - словарь n-1-грамм
    V = len(d)
    #len(ngrams_dict.get(n-1,len(unigrams))) # сглаживание лапласса: добавляем размер словаря знаменателя
    p1 = d.get(c1,0)+1
    p2 = d2.get(c2,0)+V
    
    result =  p1/p2
    
    print('\t',c1, '/', c2, ':' , p1-1, '/', p2-V, '(', p1, '/', p2, ')', '=', result)
    
    if use_log:
        return math.log(p1)-math.log(p2)
    else:
        return p1/p2
    
def context_prob(ngrams, use_log= False):
    if use_log:
        p_context = 0.0
    else:
        p_context = 1.0
    for ngram in ngrams:
        p = ngram_prob(ngram, 3, use_log= use_log)
        #print(ngram, p)
        if use_log:
            p_context+=p
        else:
            p_context*=p
    return p_context

In [64]:
df = pd.DataFrame(columns=['лемма','индекс','контекст','информативность','вероятность','близость','словарность','класс'])
df

,лемма,индекс,контекст,информативность,вероятность,близость,словарность,класс


In [98]:
def making_frame(complex_words, window=5, n=3, algorithm = ['ngram','random_three'][0]):
    def ngram_algorithm():
        fit='НЕТ'
        if p_changed_context>p_context:
            fit='ДА'
            if sub.fitness > best_fitness:
                return sub
            elif sub.fitness == best_fitness:
                if best_sub.complexity < sub.complexity:
                    return sub
                else:
                    return best_sub
            else:
                return best_sub
                    
    for token in complex_words:
        if isinstance(token, Complex_word):
            if token.easier: 
                cls = [[0]]*len(token.easier) # для таблички - классы
                in_min = [] # для таблички - словарность - в минимуме
                easier_lexems = [t.lexem for t in token.easier] # для таблички - лексемы замен
                easier_inf = [t.complexity for t in token.easier] # для таблички - сложность замен
                
                # окна контекста строим для тех сложных слов, для которых есть варианты замен проще, чем исходное
                token.make_window(complex_words, window = window) # для подсчетов
                easier_places = [[token.place]]*len(token.easier) # для таблички - место исходного
                
                context_lem = [c.lexem for c in token.context] # для подсчета вероятностей нграмм
                context_text = [c.text.lower() for c in token.context] # для таблички - контекст
                
                # определение, насколько слово близко с другими словами из контекста
                # cos_sim возвращает 0.0, если пара слов не найдена в модели
                closeness = np.mean([token.cos_sim(w) for w in token.context if w.lexem!=token.lexem])
                
                text_ngrams = [g for g in ngrams(context_lem, n)] # генерация n-грамм
                p_context = context_prob(text_ngrams, use_log=True) # вероятность контекста
                
                best_fitness = -1000  # 
                best_sub = None
                easier_prob = [] # для таблички - контексная вероятность
                easier_similarity = [] # для таблички - контексная близость
                for sub in token.easier:
                    # контекст с новым словом
                    sub_context = token.context[:token.place]+[sub]+token.context[token.place+1:]
                    # все то же самое
                    sub.closeness = np.mean([sub.cos_sim(w) for w in sub_context if w.lexem!=sub.lexem])
                    easier_similarity.append(sub.closeness) # для таблички - контексная близость
                    
                    sub_context_lem = [t.lexem for t in sub_context]
                    sub_ngrams = [g for g in ngrams(sub_context_lem, n)]
                    p_changed_context = context_prob(sub_ngrams, use_log=True)
                    sub.fitness = p_changed_context
                    
                    easier_prob.append(p_changed_context) # для таблички - контексная вероятность
                    
                    if sub in minimum:
                        in_min.append(1)
                    else:
                        in_min.append(0)
                        
                    if algorithm == 'ngram':
                        best_sub = ngram_algorithm()
                        best_fitness = best_sub.fitness
                
                if algorithm == 'ngram' and best_sub:
                    for i,sub in enumerate(token.easier):
                        if sub.lexem == best_sub.lexem:
                            cls[i]=1
                        else:
                            cls[i]=0
                    #print('Лучшее значение вероятности: {0}\n'.format(best_sub.text))
                new_lines = np.reshape(np.array([easier_lexems, easier_places, context_text, easier_inf, easier_prob, easier_similarity, in_min, cls]),(8,len(token.easier)))
                print(new_lines)

In [99]:
making_frame(complex_words)

	 бракованный белорусский лекарство / бракованный белорусский : 0 / 0 ( 1 / 2876543 ) = 3.476395103427969e-07
	 белорусский лекарство изымать / белорусский лекарство : 0 / 0 ( 1 / 2876543 ) = 3.476395103427969e-07
	 лекарство изымать из / лекарство изымать : 0 / 0 ( 1 / 2876543 ) = 3.476395103427969e-07
	 изымать из обращение / изымать из : 26 / 450 ( 27 / 2876993 ) = 9.38479864219343e-06
	 из обращение лекарственный / из обращение : 0 / 158 ( 1 / 2876701 ) = 3.4762041658135483e-07
	 бракованный белорусский лекарство / бракованный белорусский : 0 / 0 ( 1 / 2876543 ) = 3.476395103427969e-07
	 белорусский лекарство брать / белорусский лекарство : 0 / 0 ( 1 / 2876543 ) = 3.476395103427969e-07
	 лекарство брать из / лекарство брать : 0 / 0 ( 1 / 2876543 ) = 3.476395103427969e-07
	 брать из обращение / брать из : 0 / 315 ( 1 / 2876858 ) = 3.476014457439331e-07
	 из обращение лекарственный / из обращение : 0 / 158 ( 1 / 2876701 ) = 3.4762041658135483e-07


AttributeError: 'NoneType' object has no attribute 'fitness'

In [67]:
for token in complex_words:
    if isinstance(token, Complex_word):
        if token.easier: 
            # окна контекста строим для тех сложных слов, для которых есть варианты замен, которые проще, чем слово
            token.make_window(complex_words, window = 5)
            
            context_lem = [c.lexem for c in token.context]
            
            # определение, насколько слово близко с другими словами из контекста
            closeness = np.mean([token.cos_sim(w) for w in token.context if w.lexem!=token.lexem])
            
            print(token.text, token.place, ':', context_lem)
            
            # генерация n-грамм
            text_3grams = [n for n in ngrams(context_lem, 3)]
            
            print(text_3grams)
            
            p_context = context_prob(text_3grams, use_log=True)
                
            print('\nВероятность слова _{0}_ в контексте {1} = {2}, близость к контексту = {3}\n'.format(token.text, context_lem, p_context,closeness))
            
            best_fitness = -1000
            best_sub = None
            
            for sub in token.easier:
                
                # контекст с новым словом
                sub_context = token.context[:token.place]+[sub]+token.context[token.place+1:]
                
                # определение, насколько слово близко с другими словами из контекста
                sub.closeness = np.mean([sub.cos_sim(w) for w in sub_context if w.lexem!=sub.lexem])
            
                sub_context_lem = [t.lexem for t in sub_context]
                
                sub_3grams = [n for n in ngrams(sub_context_lem, 3)]
                
                p_changed_context = context_prob(sub_3grams, use_log=True)
                
                fit='НЕТ'
                if p_changed_context>p_context:
                    fit='ДА'
                    sub.fitness = p_changed_context
                    if sub.fitness > best_fitness:
                        best_fitness = sub.fitness
                        best_sub = sub
                    elif sub.fitness == best_fitness:
                        if best_sub.complexity < sub.complexity:
                            best_fitness = sub.fitness
                            best_sub = sub
                        
                print('\nВероятность замены _{0}_ в контексте {1} = {2}, разница = {3}, используем? - {4}'.format(sub.text, sub_context_lem, p_changed_context, p_context-p_changed_context, fit))
                print('Близость к контексту = {0}, разница с исходным = {1}\n'.format(sub.closeness, closeness-sub.closeness))
            if best_sub:
                print('Лучшее значение вероятности: {0}\n'.format(best_sub.text))
            

изъяли 3 : ['бракованный', 'белорусский', 'лекарство', 'изымать', 'из', 'обращение', 'лекарственный']
[('бракованный', 'белорусский', 'лекарство'), ('белорусский', 'лекарство', 'изымать'), ('лекарство', 'изымать', 'из'), ('изымать', 'из', 'обращение'), ('из', 'обращение', 'лекарственный')]
	 бракованный белорусский лекарство / бракованный белорусский : 0 / 0 ( 1 / 2876543 ) = 3.476395103427969e-07
	 белорусский лекарство изымать / белорусский лекарство : 0 / 0 ( 1 / 2876543 ) = 3.476395103427969e-07
	 лекарство изымать из / лекарство изымать : 0 / 0 ( 1 / 2876543 ) = 3.476395103427969e-07
	 изымать из обращение / изымать из : 26 / 450 ( 27 / 2876993 ) = 9.38479864219343e-06
	 из обращение лекарственный / из обращение : 0 / 158 ( 1 / 2876701 ) = 3.4762041658135483e-07

Вероятность слова _изъяли_ в контексте ['бракованный', 'белорусский', 'лекарство', 'изымать', 'из', 'обращение', 'лекарственный'] = -71.0648734045552, близость к контексту = 0.06318795363628794

	 бракованный белорусский 

поправить вычисление вероятности - в знаменателе число уникальных нграмм, а не униграмм, ну и переписать с условием n>/=1
изменить вычисление вероятности для логарифма
Написать перплексию для получившейся модели - хз как????  
поправить нахождение best_sub - если их больше 1, надо как-то это обрабатывать  
прогнать статистическую модель на текстах и сформировать таким образом корпус положительных замен, которые потом можно подредачить ручками   
заодно при редактировании ручками можно создать еще один файл, в котором редачить, а потом сравнить с первым, чтобы посчитать точность и проч  
потом все-таки посчитать 

In [ ]:
def perplexity(docs):
    docs = np.asarray(docs)
    out_of_dict = []
   
    tmp_sum_docs = 0.0
    N = 0.0
    for doc in log_progress(docs):
        tmp_sum_words = 0.0
        for word in log_progress(words):
            freq = frequencies.get(word, TOL)
            freq_empirical = frequencies_empirical.get(word, 0.0)
            N += freq_empirical
            if freq == TOL:
                out_of_dict.append(word)
            tmp_sum_words += freq_empirical * np.log(freq)
        tmp_sum_docs += tmp_sum_words
       
    return np.exp(-tmp_sum_docs / N), out_of_dict